## Tech Challenge - Fine Tuning de Foundation Model
**Aluno:** Vinícius Oliveira Litran Andrade

Neste projeto, realizo o fine-tuning de um modelo de linguagem utilizando o dataset **AmazonTitles-1.3MM**, que contém títulos de produtos e suas respectivas descrições.

O objetivo é treinar o modelo para, a partir de uma pergunta baseada no título do produto, gerar uma resposta coerente com base na descrição aprendida durante o treinamento.

Devido às limitações de hardware, o modelo utilizado foi o **DistilGPT-2**, uma versão otimizada e menor do GPT-2.

In [ ]:
# =========================================
# 📦 Importação de bibliotecas necessárias
# =========================================

import pandas as pd
from pathlib import Path
import json
import os
import re
import unicodedata
import html
from pathlib import Path
from tqdm import tqdm

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
import torch

In [ ]:
# =========================================
# 🔧 Inicialização do Tokenizer e do Modelo Base
# =========================================

model_name = 'distilgpt2'

# Carrega o tokenizer e o modelo base
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
# Adiciona token de padding se não existir
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


## Preparação dos Dados

O dataset original contém dois campos principais: 
- **Title:** Título do produto.
- **Content:** Descrição detalhada do produto.

O processo de preparação incluiu:
- Limpeza dos textos para remover caracteres não imprimíveis.
- Criação de prompts no formato: 

You are an assistant that answers questions based on the product description.

Question: {title}
Answer: {content}



In [ ]:
# =========================================
# 🧹 Funções de Pré-processamento dos Dados
# =========================================

def clean_text(text):
    """
    Limpa e normaliza texto, removendo HTML, caracteres especiais e espaços desnecessários.
    """
    if not isinstance(text, str) or not text.strip():
        return ""
    text = unicodedata.normalize("NFKC", text)                 # Normaliza acentuação
    text = re.sub(r'<[^>]+>', ' ', text)                      # Remove tags HTML
    text = html.unescape(text)                                # Converte entidades HTML (&amp;)
    text = re.sub(r'[^\x20-\x7E]+', ' ', text)                # Remove caracteres não ASCII
    text = re.sub(r'\s+', ' ', text)                          # Remove múltiplos espaços
    text = re.sub(r'[\r\n\t]', ' ', text)                     # Remove quebras de linha e tabulações
    text = re.sub(r'\s([?.!,:;"])', r'\1', text)              # Remove espaço antes de pontuação
    return text.strip()

In [5]:
def count_tokens(text):
    """
    Conta quantos tokens o texto possui de acordo com o tokenizer.
    """
    return len(tokenizer.encode(text, add_special_tokens=False))

In [6]:
def prepare_data(input_path, output_path, min_tokens=10, max_total_tokens=256, percent_dataset=0.1):
    """
    Processa o dataset JSONL original e gera um dataset formatado com perguntas e respostas.
    """
    output_file = Path(output_path)
    num_written = 0

    with open(input_path, 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)  # Conta total de linhas (exemplos)

    max_examples = int(total_lines * percent_dataset)  # Define amostragem

    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8') as outfile:

        for line in tqdm(infile, desc="Processando"):
            if num_written >= max_examples:
                break

            item = json.loads(line)
            title = clean_text(item.get('title', ''))
            content = clean_text(item.get('content', ''))

            if not title or not content:
                continue

            # Formata prompt de maneira mais clara para o modelo
            text = f"""
            You are an assistant that answers questions based on the product description.

            Question: {title}
            Answer: {content}
            """.strip()

            num_tokens = count_tokens(text)

            if num_tokens < min_tokens:
                continue
            if num_tokens > max_total_tokens:
                continue

            example = {"text": text}
            outfile.write(json.dumps(example, ensure_ascii=False) + '\n')
            num_written += 1

    print(f"\n✅ Dados salvos em {output_file} com {num_written} exemplos.")

In [ ]:
# =========================================
# 🚀 Preparação e Tokenização do Dataset
# =========================================

# Executa a preparação dos dados
prepare_data(
    input_path='LF-Amazon-1.3M/trn.json',
    output_path='fine_tune_data.jsonl',
    min_tokens=5,
    max_total_tokens=256
)

# Carrega dataset no formato JSONL
dataset = load_dataset('json', data_files={'train': 'fine_tune_data.jsonl'})

Processando: 525it [00:00, 5236.46it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1127 > 1024). Running this sequence through the model will result in indexing errors
Processando: 558809it [02:39, 3506.46it/s] 



✅ Dados salvos em fine_tune_data.jsonl com 224861 exemplos.


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# =========================================
# ✍️ Tokenização
# =========================================

# Função para tokenização dos dados
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=256
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Para Language Modeling, entrada = saída
    return tokenized

# Aplica tokenização
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text'],
    num_proc=1
)

Map:   0%|          | 0/224861 [00:00<?, ? examples/s]

## 🎯 Fine-Tuning do Modelo

O fine-tuning foi realizado utilizando o modelo **DistilGPT-2**, uma versão otimizada e compacta do GPT-2, que possui aproximadamente **82 milhões de parâmetros**, em comparação com os **124 milhões do GPT-2 original**.

---

### 🔍 Justificativa da Escolha do Modelo

O modelo **DistilGPT-2** foi selecionado por ser:

- ⚡ **Leve e rápido de treinar.**  
- 💻 **Menos demandante em termos de recursos computacionais.**  
- 🏠 **Viável para execução local em máquinas com GPUs de consumo**, como **RTX 4070**, **RTX 3060** ou **RTX 2060**.  
- 🔄 **Capaz de executar todo o ciclo de:**
  - **Pré-processamento**
  - **Treinamento (fine-tuning)**
  - **Geração de respostas**

➡️ Tudo isso **sem custos adicionais com infraestrutura em nuvem**, dentro de prazos compatíveis com as demandas acadêmicas.

> ⚠️ Embora o desempenho não seja comparável a modelos maiores, como **Llama 2**, **Mistral** ou **GPT-3**, ele é plenamente suficiente para **validar o pipeline completo de fine-tuning aplicado ao dataset _AmazonTitles-1.3MM_**.

---

## 🔧 Parâmetros do Fine-Tuning

| **Parâmetro**         | **Valor**            | **Justificativa**                                                            |
|-----------------------|----------------------|------------------------------------------------------------------------------|
| **Modelo base**       | `distilgpt2`         | Leve, otimizado, adequado para rodar localmente                              |
| **Batch Size**        | `16`                 | Equilíbrio entre consumo de memória e velocidade de treinamento              |
| **Épocas (Epochs)**   | `3`                  | Reduz tempo de treino e risco de overfitting                                 |
| **Max Tokens**        | `256`                | Limita o contexto ao tamanho que o modelo suporta                            |
| **Learning Rate**     | `3e-4`               | Valor padrão eficaz para modelos autoregressivos                             |
| **Weight Decay**      | `0.01`               | Regularização que ajuda a prevenir overfitting                               |
| **FP16 Precision**    | `True` (se CUDA)     | Acelera o treino e reduz consumo de memória se houver GPU                    |
| **Save Steps**        | `500`                | Salva checkpoints a cada 500 passos                                          |
| **Save Total Limit**  | `1`                  | Mantém apenas o checkpoint mais recente para economizar espaço               |
| **Logging Steps**     | `100`                | Frequência dos logs durante o treinamento                                    |

---

## 🚩 Observações Importantes

- O uso de **apenas 3 épocas** foi suficiente, considerando:
  - 📊 O tamanho do modelo.
  - 🔧 A simplicidade da tarefa (gerar respostas a partir de descrições).
  - ⚠️ O risco de **overfitting**, que cresce rapidamente em modelos menores com mais épocas.

- Para tarefas mais complexas ou modelos maiores, seriam necessárias:
  - 🔼 Mais épocas de treinamento.
  - ☁️ Recursos em cloud computing.
  - 🎯 Ajustes mais refinados nos hiperparâmetros.


In [ ]:
# =========================================
# ⚙️ Configuração do Fine-Tuning
# =========================================

output_dir = "fine_tuned_model"

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # False porque GPT-2 é um modelo causal, não masked
)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    learning_rate=5e-5,  # Valor mais estável para esse tipo de tarefa
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # Usa meia precisão se houver GPU
    push_to_hub=False,
    report_to="none",
    optim="adamw_torch_fused",  # Otimizador mais moderno e eficiente
)

In [ ]:
# =========================================
# 🏋️ Execução do Fine-Tuning
# =========================================

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    data_collator=data_collator
)

# Move modelo para GPU, se disponível
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

In [11]:
# Inicia treinamento
trainer.train()

# Salva modelo e tokenizer treinados
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Modelo fine-tuned salvo em {output_dir}")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.062100
200,2.881900
300,2.859400
400,2.825800
500,2.821000
600,2.782300
700,2.784700
800,2.774700
900,2.754000
1000,2.716700


✅ Modelo fine-tuned salvo em fine_tuned_model


In [12]:
# =========================================
# 🔗 Função para Carregar o Modelo Treinado
# =========================================

def load_model(model_path):
    """
    Carrega o modelo e tokenizer treinados a partir do diretório.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)
    model.eval()
    return tokenizer, model


## Geração de Respostas

Após o treinamento, o modelo foi configurado para receber perguntas no seguinte formato:

```plaintext
Pergunta: {título do produto}
Resposta:


In [13]:
# =========================================
# 🗣️ Função de Geração de Respostas
# =========================================

def gerar_resposta(pergunta, tokenizer, model, max_length=150):
    """
    Gera uma resposta a partir de uma pergunta fornecida.
    """
    prompt = f"""
    You are an assistant that answers questions based on the product description.

    Question: {pergunta}
    Answer:
    """.strip()

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )

    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta = resposta.split("Resposta:")[-1].strip()

    return resposta

In [15]:
# 🔍 Função de teste do modelo (antes e depois do fine-tuning)

def testar_modelo(pergunta, tokenizer, model, max_length=150):
    """
    Gera uma resposta para a pergunta fornecida.
    Funciona tanto antes quanto depois do fine-tuning.
    """
    prompt = f"Pergunta: {pergunta}\nResposta:"
    
    inputs = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True
    ).to(model.device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extrai apenas o conteúdo após "Resposta:"
    if "Resposta:" in resposta:
        resposta = resposta.split("Resposta:")[-1]
    if "Pergunta:" in resposta:
        resposta = resposta.split("Pergunta:")[0]

    return resposta.strip()


In [16]:
perguntas = [
    # Pergunta baseada em "Girls Ballet Tutu Neon Pink" ou "Girls Ballet Tutu Neon Blue"
    "Tell me about the Girls Ballet Tutu Neon Pink.", 
    # Pergunta baseada em "Mog's Kittens"
    "What kind of book is Mog's Kittens?",
    # Pergunta baseada em "The Prophet"
    "Can you describe The Prophet by Gibran?",
    # Pergunta baseada em "Spirit Led-Moving By Grace In The Holy Spirit's Gifts"
    "What is the content of Spirit Led-Moving By Grace In The Holy Spirit's Gifts?",
    # Pergunta baseada em "Mexico The Beautiful Cookbook" ou "America: The Beautiful Cookbook"
    "What is special about the Mexico The Beautiful Cookbook?",
]

In [17]:
print("🧠 🔍 Respostas ANTES do fine-tuning:\n")

for pergunta in perguntas:
    resposta = testar_modelo(pergunta, tokenizer, model)
    print(f"🔸 Pergunta: {pergunta}")
    print(f"🔹 Resposta antes do fine-tuning: {resposta}\n")


🧠 🔍 Respostas ANTES do fine-tuning:

🔸 Pergunta: Tell me about the Girls Ballet Tutu Neon Pink.
🔹 Resposta antes do fine-tuning: The Only Time I Ever Had to Get a Job
This book has been a labor of love for girls since I was a child. I have been the best job I've ever had to get a job since reading this book. If you are interested in girls and girls, this is the book for you. --The Author, Author of the Best Girls' Books Ever! I am excited to see how you have helped me with this wonderful book! --Marianne St. John, Founder and President, Inc.I wish I had this one in my books. It is an inspiring book that I would have loved to read. Thank you

🔸 Pergunta: What kind of book is Mog's Kittens?
🔹 Resposta antes do fine-tuning: The Complete Guide to the Mog Mog
Mog: A Complete Book of Mog (Volume 1)
The Mog: Complete Series: Mog, a Complete Volume 1 (also called the Complete Mog Series) (Mogg, the Ultimate Guide) and Mog Guide (The Ultimate Guides to Mog). The book includes over 50 maps and m

In [18]:
# Carrega o modelo fine-tuned
tokenizer_finetuned, model_finetuned = load_model('fine_tuned_model')

print("🧠 🚀 Respostas DEPOIS do fine-tuning:\n")

for pergunta in perguntas:
    resposta = testar_modelo(pergunta, tokenizer_finetuned, model_finetuned)
    print(f"🔸 Pergunta: {pergunta}")
    print(f"🔹 Resposta depois do fine-tuning: {resposta}\n")


🧠 🚀 Respostas DEPOIS do fine-tuning:

🔸 Pergunta: Tell me about the Girls Ballet Tutu Neon Pink.
🔹 Resposta depois do fine-tuning: The Story of a Novel

The story of the storyteller at work in the 1950s and 60s
A new book by a young girl at the age of nine tells of her first day as a girl and her love for art and music. --Rendezvous, September, 2001From the Foreword by Lesley P. Smith, aNew York Timesbestselling author of The Lady Who Wrote The Book of My LifeA gripping story about a talented young woman who was the first woman to earn a living and was a successful artist. In her debut novel, The Girl Who Readed the Book Of My Dreams, she

🔸 Pergunta: What kind of book is Mog's Kittens?
🔹 Resposta depois do fine-tuning: A Very Short Introduction
SOG's Masks in the World of Black Catacombs
The book's first section is a primer for the Moges and other related subjects in this series, with a glossary of terms and a detailed index. The second section covers the history and origin of the Kit

In [20]:
# =========================================
# 🧠 Interação com o Modelo Treinado
# =========================================

model_path = 'fine_tuned_model'

if not os.path.exists(model_path):
    raise FileNotFoundError(
        f"A pasta {model_path} não foi encontrada. Execute o fine-tuning primeiro."
    )

tokenizer, model = load_model(model_path)

print("Modelo carregado. Você pode começar a fazer perguntas. Digite 'sair' para encerrar.")

while True:
    pergunta = input("Faça sua pergunta: ")
    if pergunta.lower() == 'sair':
        print("Encerrando o gerador de respostas.")
        break
    resposta = gerar_resposta(pergunta, tokenizer, model)
    print(f"Resposta: {resposta}")

Modelo carregado. Você pode começar a fazer perguntas. Digite 'sair' para encerrar.
Resposta: You are an assistant that answers questions based on the product description.

    Question: The Best of India: A Cookbook
    Answer: A cookbook to help you cook and drink as well as enjoy the pleasures of India. --Greece Times, January 2009In this cookbook, Indian food expert Darshan Pradeep offers an array of recipes to help you discover and enjoy your own India, including the best of India. --Indian Express, January 2009In this cookbook, Indian food expert Darshan Pradeep offers an array of recipes to help you discover and enjoy your own India, including the best of India. --Indian Express, January 2009In this cookbook, Indian food expert D
Encerrando o gerador de respostas.



---

### 🔹 **Conclusão**
```markdown
## Conclusão

Este projeto demonstrou, na prática, o processo de fine-tuning de um modelo de linguagem baseado no dataset AmazonTitles-1.3MM. Mesmo utilizando um modelo leve como o DistilGPT-2, foi possível observar melhorias claras na geração de respostas após o treinamento.

Para aplicações em produção ou cenários mais exigentes, recomenda-se utilizar foundation models mais robustos, como os da família **Llama** ou **Mistral**.

O código completo e a demonstração podem ser acessados nos links fornecidos no arquivo de entrega ou no github: https://github.com/UnB-EngEnerg-180028863/Terceiro-Tech-Challenge.